In [3]:
import os
import numpy as np
import pandas as pd
from scipy.io import loadmat
from sklearn.ensemble import RandomForestClassifier
import random

# 指定文件夹路径
download_save_path = 'E:/Dataset/wind_shear/Data_Download'
exception_save_path = '../Dataset/Exception_Data'
plt_save_path = '../result/WSHR/figures'

# 获取文件夹下的所有文件名称
dowload_folder_names = [item for item in os.listdir(download_save_path) if os.path.isdir(os.path.join(download_save_path, item))]
exception_folder_names = [item for item in os.listdir(exception_save_path) if os.path.isdir(os.path.join(exception_save_path, item))]
instruction_folder_names = ["@Instructions"]

# 生成所有文件夹路径
dowload_folder_paths = [os.path.join(download_save_path, item) for item in dowload_folder_names]
exception_folder_paths = [os.path.join(exception_save_path, item) for item in exception_folder_names]

In [1]:
# give the preset classification of variables
group_names_list = ["mechanism", "power", "control", "external", "recorder", "unclassified"]
group_lens_dict = {}

var_group_mechanism = ["AIL_1", "AIL_2", "FLAP", "ELEV_1", "ELEV_2", "RUDD", "SPL_1", "SPL_2", "SPLG", "SPLY", "ABRK", "BPGR_1", "BPGR_2", "BPYR_1", "BPYR_2", "MSQT_1", "MSQT_2", "NSQT", "BLV", "CALT", "PACK", "WOW", 
                       "AOA1", "AOA2", "GLS", "PTCH", "ROLL", 
                       "TH", "MH", "TAS", "CASM", "GS", "IVV",
                       "VRTG", "LATG", "LONG", "FPAC", "CTAC"]
var_group_power = ["N2_1", "N2_2", "N2_3", "N2_4",
                   "ECYC_1", "ECYC_2", "ECYC_3", "ECYC_4", "EHRS_1", "EHRS_2", "EHRS_3", "EHRS_4", "VIB_1", "VIB_2", "VIB_3", "VIB_4", "FADS", "HYDG", "HYDY",
                   "N1_1", "N1_2", "N1_3", "N1_4", "N1T", "FF_1", "FF_2", "FF_3", "FF_4", "FQTY_1", "FQTY_2", "FQTY_3", "FQTY_4", "OIP_1", "OIP_2", "OIP_3", "OIP_4", "OIT_1", "OIT_2", "OIT_3", "OIT_4", "OIPL", "EGT_1", "EGT_2", "EGT_3", "EGT_4",
                   "LGDN", "LGUP"]
var_group_control = ["CRSS", "HDGS", "A_T", "APFD", "DFGS", "FGC3", "PUSH", "PTRM", "TCAS",
                     "ILSF", "RUDP", "CCPC", "CCPF", "CWPC", "CWPF", "PLA_1", "PLA_2", "PLA_3", "PLA_4",
                     "SNAP", "TMODE", "EAI", "TAI", "WAI_1", "WAI_2", 
                     "APUF", "FADF", "FIRE_1", "FIRE_2", "FIRE_3", "FIRE_4", "GPWS", "MW", "POVT", "SHKR", "SMOK", "TOCW"]
var_group_external = ["ALT", "ALTR", "WS", "WD", "PI", "PS", "PT", "SAT", "TAT",
                      "DA", "TRK", "TRKM", "LOC", "LATP", "LONP"]
var_group_recorder = ["DWPT", "PH", 
                     "ACMT", "FRMC", "GMT_HOUR", "GMT_MINUTE", "GMT_SEC"]
var_group_unclassified = ["ATEN", "EVNT", "HF1", "HF2", "VHF1", "VHF2", "VHF3", "LMOD", "VMODE", "MACH", "MNS", "MRK", "N1C", "N1CO", "SMKB", "VAR_1107", "VAR_2670", "VAR_5107", "VAR_6670"]

var_groups_dict = {"mechanism": var_group_mechanism, "power": var_group_power, "control": var_group_control, "external": var_group_external, "recorder": var_group_recorder, "unclassified": var_group_unclassified}
for group_name, var_group in var_groups_dict.items():
    group_lens_dict[group_name] = len(var_group)
    print(f"{group_name}: {len(var_group)}")
print(f"\n{sum(group_lens_dict.values())} variables in total")

# 给定数据不完整的变量列表
invalid_var_dict = {'mechanism': ['FLAP', 'SPLG', 'SPLY', 'BPGR_1', 'BPGR_2', 'BPYR_1', 'BPYR_2', 'MSQT_1', 'MSQT_2', 'NSQT', 'BLV', 'CALT', 'PACK', 'WOW', 'AOA1', 'AOA2', 'GLS', 'PTCH', 'ROLL', 'TH', 'MH', 'TAS', 'CASM', 'GS', 'IVV', 'FPAC', 'CTAC'], \
                    'power': ['N2_1', 'N2_2', 'N2_3', 'N2_4', 'ECYC_1', 'ECYC_2', 'ECYC_3', 'ECYC_4', 'EHRS_1', 'EHRS_2', 'EHRS_3', 'EHRS_4', 'VIB_1', 'VIB_2', 'VIB_3', 'VIB_4', 'FADS', 'HYDG', 'HYDY', 'N1_1', 'N1_2', 'N1_3', 'N1_4', 'N1T', 'FF_1', 'FF_2', 'FF_3', 'FF_4', 'FQTY_1', 'FQTY_2', 'FQTY_3', 'FQTY_4', 'OIP_1', 'OIP_2', 'OIP_3', 'OIP_4', 'OIPL', 'EGT_1', 'EGT_2', 'EGT_3', 'EGT_4', 'LGDN', 'LGUP'], \
                        'control': ['CRSS', 'HDGS', 'A_T', 'APFD', 'DFGS', 'FGC3', 'PUSH', 'TCAS', 'RUDP', 'CCPC', 'CCPF', 'CWPC', 'CWPF', 'PLA_1', 'PLA_2', 'PLA_3', 'PLA_4', 'SNAP', 'TMODE', 'EAI', 'TAI', 'WAI_1', 'WAI_2', 'APUF', 'FADF', 'FIRE_1', 'FIRE_2', 'FIRE_3', 'FIRE_4', 'MW', 'POVT', 'SHKR', 'SMOK', 'TOCW'], \
                            'external': ['ALT', 'ALTR', 'WS', 'WD', 'PI', 'PS', 'PT', 'SAT', 'TAT', 'DA', 'TRK', 'TRKM', 'LOC', 'LATP'], \
                                'recorder': ['DWPT', 'PH', 'GMT_HOUR', 'GMT_MINUTE', 'GMT_SEC'], \
                                    'unclassified': ['ATEN', 'EVNT', 'HF1', 'HF2', 'VHF1', 'VHF2', 'VHF3', 'LMOD', 'VMODE', 'MACH', 'MNS', 'MRK', 'N1C', 'N1CO', 'SMKB']}

valid_var_dict = {group_name: list(set(var_groups_dict[group_name]) - set(invalid_var_dict[group_name])) for group_name in var_groups_dict.keys()}

# 查找给定总序数对应的变量名称
def find_var_name(idx):
    count = 0
    for group_name, var_group in var_groups_dict.items():
        if count + group_lens_dict[group_name] > idx:
            return group_name, var_group[idx - count]
        else:
            count += group_lens_dict[group_name]

mechanism: 38
power: 47
control: 37
external: 15
recorder: 7
unclassified: 19

163 variables in total


In [38]:
find_var_name(100)

('control', 'PLA_1')

In [4]:
# 读取一个异常mat文件
e_mat_path = os.path.join(exception_folder_paths[0], os.listdir(exception_folder_paths[0])[0])
e_mat = loadmat(e_mat_path)

# 初始化解释变量和响应变量的存储array
explain_var_array, response_wshr_array = [], []

# 分别读取自变量和WSHR因变量
response_wshr_array = e_mat["WSHR"][0][0][0]

# for group_name, var_list in var_groups_dict.items():
for group_name, var_list in valid_var_dict.items():
    for var_name in var_list:
        # 对每个变量按照rate进行下采样或过采样
        var_data, var_rate = e_mat[var_name][0][0][0], e_mat[var_name][0][0][1][0][0]
        if var_rate == 1:
            explain_var_array.append(var_data)
        elif var_rate > 1: # 进行下采样
            # print(len(var_data.tolist()))
            explain_var_array.append(random.sample(var_data.tolist(), k=len(response_wshr_array)))
        else:
            explain_var_array.append(random.choices(var_data, k=len(response_wshr_array)))

explain_var_array = np.squeeze(np.array(explain_var_array)).T
response_wshr_array = np.array(response_wshr_array)

print(explain_var_array.shape)
print(response_wshr_array.shape)

(1252, 25)
(1252, 1)


In [5]:
# 创建随机森林分类器
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# 拟合模型
rf.fit(explain_var_array, response_wshr_array)

# 获取特征重要性并排序输出
feature_importances = rf.feature_importances_
importances_indices = np.argsort(feature_importances)[::-1]
top_k = explain_var_array.shape[1]  # 选择前 k 个重要性最高的特征
feature_names = [f"Feature {i}: {find_var_name(i)}" for i in range(explain_var_array.shape[1])]

print("Top", top_k, "Features:")
for i in range(top_k):
    print(f"{i+1}. {feature_names[importances_indices[i]]} ({feature_importances[importances_indices[i]]})")


Top 25 Features:
1. Feature 2: ('mechanism', 'FLAP') (0.18728756381768843)
2. Feature 9: ('mechanism', 'SPLY') (0.17378979969603303)
3. Feature 6: ('mechanism', 'SPL_1') (0.08539396297892134)
4. Feature 19: ('mechanism', 'CALT') (0.06690690059225032)
5. Feature 8: ('mechanism', 'SPLG') (0.06442082556898619)
6. Feature 4: ('mechanism', 'ELEV_2') (0.0644096725703083)
7. Feature 14: ('mechanism', 'BPYR_2') (0.054608354593199175)
8. Feature 0: ('mechanism', 'AIL_1') (0.054523932066631885)
9. Feature 7: ('mechanism', 'SPL_2') (0.04962075887550051)
10. Feature 1: ('mechanism', 'AIL_2') (0.04811360269031083)
11. Feature 20: ('mechanism', 'PACK') (0.03734953286296099)
12. Feature 3: ('mechanism', 'ELEV_1') (0.027890569266937042)
13. Feature 15: ('mechanism', 'MSQT_1') (0.02664358505820865)
14. Feature 5: ('mechanism', 'RUDD') (0.025704414563157328)
15. Feature 12: ('mechanism', 'BPGR_2') (0.013797748871025466)
16. Feature 11: ('mechanism', 'BPGR_1') (0.010469708847113741)
17. Feature 13: ('mec

C:\Users\86108\AppData\Local\Temp\ipykernel_148304\3828022564.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(explain_var_array, response_wshr_array)
